## Loan Performance - CJ 

This notebook practices the Classification - KNN (K-th nearest neighbor of learning) for the Week 6 Capstone project within IBM - Machine Learning with Python 
File Start: 09/28/2019 11am ET 
Author: Chelsea Jiang 

Data Processing

In [ ]:
## Import Coding Packages 
import itertools
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import NullFormatter
import pandas as pd
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics 
%matplotlib inline

In [2]:
# Download Dataset 
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2019-09-29 15:30:59--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.002s  

2019-09-29 15:30:59 (12.4 MB/s) - ‘loan_train.csv’ saved [23101/23101]



### Load Data from CSV File (df)

In [3]:
# Load Data from the CSV File - df 
df = pd.read_csv('loan_train.csv')

### Process Data from CSV File (df)

In [4]:
## Data Processing - Convert Date time object (effective date and due date) - *** KEEPING Column names!
df['effective_date'] = pd.to_datetime(df['effective_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

In [5]:
## Data Processing - Day of week (by effective date) 
## Categorize by "weekend = 1" (vs. weekday)
df['dayofweek'] = df['effective_date'].dt.dayofweek 
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)

In [6]:
## Data Processing - To numerical values (gender coding: FEMALE = 1)
## do NOT rerun - ONLY RUN THIS ONCE, because it is using Replace 
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)

In [7]:
## Data Processing - One hot coding Education (drop one, correct typo) 
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.rename(columns={'Bechalor':'Bachelor'},inplace = True)
Feature.head()

,Principal,terms,age,Gender,weekend,Bachelor,High School or Below,college
0,1000,30,45,0,0,0,1,0
1,1000,30,33,1,0,1,0,0
2,1000,15,27,0,0,0,0,1
3,1000,30,28,1,1,0,0,1
4,1000,30,29,0,1,0,0,1


In [8]:
## Data Processing - Feature Selection (define X (feature set), y (loan_status))
X = Feature
y = df['loan_status'].values

In [9]:
## Data Normalization (norm/fit X)
X = preprocessing.StandardScaler().fit(X).transform(X)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [10]:
## Train Test Split 
## change Test size selection? not 0.2? 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)                        

Train set: (276, 8) (276,)
Test set: (70, 8) (70,)


## Classification - K nearest neighbor (KNN)

In [11]:
## KNN - Import Library 
from sklearn.neighbors import KNeighborsClassifier

## KNN - Training and plotting best K for each accuracy evaluation  
Ks = 20
JC = np.zeros((Ks-1))
F1 = np.zeros((Ks-1))
for k in range(1,Ks):
    nb  = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    yhat = nb.predict(X_test)
    JC[k-1] =  metrics.jaccard_similarity_score(y_test, yhat, normalize=True)
    F1[k-1] = metrics.f1_score(y_test, yhat,average = 'macro')
print('By Jaccard Index, the best accuracy is', round(JC.max(),5), 'with K =', JC.argmax())
print('By F1-Score, the best accuracy is', round(F1.max(),5), 'with K =', F1.argmax())

## (??) Log loss is not applicable as the result is categorical instead of continuous probability? 

By Jaccard Index, the best accuracy is 0.78571 with K = 6
By F1-Score, the best accuracy is 0.66298 with K = 7


## Classification - Decision Tree 

In [12]:
## Decision Tree - Import Library 
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 3)
#print ('Train set:', X_train.shape,  y_train.shape)
#print ('Test set:', X_test.shape,  y_test.shape)   
## Creating an instance of the decision tree DT and Train it 

DT = DecisionTreeClassifier(criterion="entropy", max_depth = 5)
DT.fit(X_train, y_train)
## Predictions using the tree 
PT = DT.predict(X_test)

## Accuracy of the Tree 
JC =  metrics.jaccard_similarity_score(y_test, yhat, normalize=True)
F1 = metrics.f1_score(y_test, yhat,average = 'macro')
print('By Jaccard Index, accuracy of the decision tree is', round(JC ,5))
print('By F1-Score, accuracy of the decision tree is,', round(F1,5))

By Jaccard Index, accuracy of the decision tree is 0.61429
By F1-Score, accuracy of the decision tree is, 0.48571


## Support Vector Machine

In [ ]:
X = np.asarray(Feature)
y = np.asarray(df['loan_status'])
y[0:5]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
## Testing different SVM kernels 
# RBF 
from sklearn import svm
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel)
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)

    ## Accuracy of the Tree 
    JC =  metrics.jaccard_similarity_score(y_test, yhat, normalize=True)
    F1 = metrics.f1_score(y_test, yhat,average = 'macro')
    print('By Jaccard Index, accuracy of the SVM with Kernel ',kernel,  'is', round(JC ,5))
    print('By F1-Score, accuracy of the SVM with Kernel ', kernel,  'is', round(F1 ,5))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


By Jaccard Index, accuracy of the SVM with Kernel  linear is 0.78571
By F1-Score, accuracy of the SVM with Kernel  linear is 0.44


In [ ]:
# Linear

In [ ]:
# Polynomial 

In [ ]:
# Sigmoid 